## Import Important LIB

In [48]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import json
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI #ต้องเป็นAzureChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [49]:
from azure.ai.inference.models import (
    ChatCompletionsToolDefinition,
    FunctionDefinition,
    UserMessage,
)

## Test api calling

In [50]:
import requests

# Define the FastAPI base URL
BASE_URL = "http://127.0.0.1:8000"  # Change if hosted elsewhere

# Search for books with "Harry" in the title
query = "Harry"
response = requests.get(f"{BASE_URL}/items/", params={"query": query})

# Check if the request was successful
if response.status_code == 200:
    items = response.json()
    print(items,type(items))
    if items:
        print("Books found:")
        for item in items:
            print(f"ID: {item['id']}, Title: {item['Title']}, Price: {item['Price']}")
    else:
        print("No books found with the given name.")
else:
    print(f"Error: {response.status_code}, {response.text}")


[{'Title': 'Harry potter', 'Description': 'magical school and education beginner to flying bloom', 'Price': 999.0, 'Type': 'Fantasy', 'InStock': 47, 'id': 1}] <class 'list'>
Books found:
ID: 1, Title: Harry potter, Price: 999.0


## Define Function call > Tools

In [51]:
from langchain.tools import tool

In [52]:
@tool
def get_overall_temp(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "32",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [53]:
import requests
@tool
def get_book_detail(book_name,unit='Baht'):
    """Get the book detail like price and amount of book in the stock """
    BASE_URL = "http://127.0.0.1:8000"
    response = requests.get(f"{BASE_URL}/items/", params={"query": book_name})
    if response.status_code == 200:
        items = response.json()
        if items:
            print("Books found:")
            for item in items:
                print(f"ID: {item['id']}, Title: {item['Title']}, Price: {item['Price']}")
            item_title=items[0]['Title']
            item_instock=items[0]['InStock']
            item_price=items[0]['Price']
        else:
            print("No books found with the given name.")
            
    else:
        print(f"Error: {response.status_code}, {response.text}")
                # book_detail={
                #     "title": item['id'],
                #     "type":f"{response.type}",
                #     "instock":f"{response.Instock}",
                #     "price":f"{response.price}"
                # }
    return f'The book {item_title} are currently {item_instock} available instock \n the price of this book is {item_price}'

## Chat Template

In [54]:
# get_temp = ChatCompletionsToolDefinition(
#     function=FunctionDefinition(
#         name="get_temp",
#         description="""Return Overall temperature of that location """,
#         parameters={
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "Unit": {
#                     "type": "string",
#                     "enum": ["celsius", "fahrenheit"],
#                 },
#             },
#             "required": ["location"],
#         },
#     )
# )

In [55]:
# get_book_info = ChatCompletionsToolDefinition(
#     function=FunctionDefinition(
#         name="get_book_detail",
#         description="""Return description of that book """,
#         parameters={
#             "type": "object",
#             "properties": {
#                 "Bookname": {
#                     "type": "string",
#                     "description": "The unique name that is book name",
#                 },
#                 "Unit": {
#                     "type": "string",
#                     "enum": ["baht"]
#                 },
#             },
#             "required": ["Bookname"],
#         },
#     )
# )

## Tools

In [56]:
tools=[get_book_detail,get_overall_temp]

## Chain

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

Convert tools to openai format

In [58]:
functions = [format_tool_to_openai_function(f) for f in tools]

## Model initial

In [59]:
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
token = os.environ["GITHUB_TOKEN"]

Note : การสร้างModel OpenAI จาก Azure ต้องทำตามแบบนี้ พวกparameterจุกจิกพวกนี้หมดเลย

In [124]:
# Azzure เขียนการinit model แบบนี้ไม่ได้/ ลองเปลี่ยนเป็น AzureChatOpenAI ก็ไม่ได้
# model = ChatOpenAI(temperature=0).bind(functions=functions)
model = AzureChatOpenAI(
    openai_api_key=token,
    openai_api_base=endpoint,
    model=model_name,
    openai_api_version="2023-03-15-preview",
    temperature=0
).bind(functions=functions)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://models.inference.ai.azure.com to https://models.inference.ai.azure.com/openai.
  warnings.warn(


In [125]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [126]:
result.tool

'get_overall_temp'

In [127]:
result.tool_input

{'location': 'San Francisco'}

In [128]:
result = chain.invoke({"input": "I want more information of book name Wan thong"})
result

AgentActionMessageLog(tool='get_book_detail', tool_input={'book_name': 'Wan thong'}, log="\nInvoking: `get_book_detail` with `{'book_name': 'Wan thong'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"book_name":"Wan thong"}', 'name': 'get_book_detail'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f6a330e0-a8b8-4e8e-a19e-c8a139ac71ba-0')])

In [129]:
result.tool

'get_book_detail'

In [130]:
result.tool_input

{'book_name': 'Wan thong'}

In [100]:
result=chain.invoke('Hi')
result.return_values['output']

'Well, hello there! What can I do for you today?'

<span style="color:#fc4c75">บรรทัดด้านล่างควรรันแล้ว error เพราะมันเรียกดูTool แต่คำสั่งไม่จำเป็นต้องใช้Tool</span>

In [102]:
result.tool

AttributeError: 'AgentFinish' object has no attribute 'tool'

In [131]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [132]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [134]:
# what is the weather is sf
# I want more information of book name Wan thong
result1 = chain.invoke({
    "input": "I want more information of book name Wan thong",
    "agent_scratchpad": []
})
print(result1)

tool='get_book_detail' tool_input={'book_name': 'Wan thong'} log="\nInvoking: `get_book_detail` with `{'book_name': 'Wan thong'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"book_name":"Wan thong"}', 'name': 'get_book_detail'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0f7eab1e-aeaa-454b-8e94-9a16ec68dc85-0')]


In [135]:
# observation = get_overall_temp(result1.tool_input)
observation = get_book_detail(result1.tool_input)
observation

Books found:
ID: 2, Title: Wan Thong, Price: 20.0


'The book Wan Thong are currently 888 available instock \n the price of this book is 20.0'

In [136]:
from langchain.agents.format_scratchpad import format_to_openai_functions
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"book_name":"Wan thong"}', 'name': 'get_book_detail'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5154047bf2', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0f7eab1e-aeaa-454b-8e94-9a16ec68dc85-0'),
 FunctionMessage(content='The book Wan Thong are currently 888 available instock \n the price of this book is 20.0', additional_kwargs={}, response_metadata={}, name='get_book_detail')]

In [138]:
# what is the weather is sf?
# I want more information of book name Wan thong
result2 = chain.invoke({
    "input": "I want more information of book name Wan thong", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})
result2

AgentFinish(return_values={'output': 'The book "Wan Thong" is currently in stock with 888 copies available. You can grab it for just 20 Baht. Happy reading! 📚✨'}, log='The book "Wan Thong" is currently in stock with 888 copies available. You can grab it for just 20 Baht. Happy reading! 📚✨')

<span style="color:#dbbd5a">จากข้างบน สามารถสรุปได้ว่า</span> 
1. พอเราส่ง prompt ของเราเข้าไป มันจะเอาไปประมวลผ่านLLMก่อนว่าจะเรียกใช้ Agent/Tools ดีไหม
2. สมมุติว่ามันไม่จำเป็นต้องเรียกใช้ Agent มันจะเข้าสู่ State AgentFinish ซึ่งแปลว่าถ้าเรา print result ก็จะได้สิ่งที่มันไปประมวลผ่าน LLMมาเลยทันที
3. ถ้ามันเกิดว่าต้องใช้ Agent มันจะมีพวกKeyword ที่ทำให้มันรู้ว่า ต้องใช้Keyword นี้แหละ ไปเรียกใช้ LLM จากการDefine tools ขอวงเรา เช่นพวกชื่อสถานที่ ชื่อหนังสือ มันก็จะส่งKeyword พวกนี้เข้าไปในStateถัดไป
4. Stateนี้ LLM จะเอาสิ่งที่สกัดมา มาเป็นInput เราเลยต้องสร้่าง Input อีกที โดยผลการประมวลครั้งแรกว่าต้องใช้function call อะไรและfunctioncallจะมีการรับinputอะไรไปบ้าง 
ถูกเก็บในตัวแปล agent_scratchpad
5. เรียกใช้ func. call เพื่อให้ได้ observation ,ผลจากการรัน function call
6. format_to_openai_functions เป็นการเปลี่ยนผลการรันเป็นสิ่งที่ LLM จะตอบกลับไปเป็นภาษาคนได้

## Auto Selection Tools 

เป็นการสร้าง funciton เพื่อให้ return ผลการประมวลจาก LLM ถ้ามันเข้าphase AgentFinish แต่ถ้ายังก็เข้า tools จนกว่าจะได้ผล

In [119]:
# ตัวนี้ไม่ได้ใช้
# from langchain.schema.agent import AgentFinish
# def run_agent(user_input):
#     intermediate_steps = []
#     while True:
#         result = chain.invoke({
#             "input": user_input, 
#             "agent_scratchpad": format_to_openai_functions(intermediate_steps)
#         })
#         if isinstance(result, AgentFinish):
#             return result
#         tool = {
#             "get_book_detail": get_book_detail, 
#             "get_overall_temp": get_overall_temp,
#         }[result.tool]
#         observation = tool.run(result.tool_input)
#         intermediate_steps.append((result, observation))

In [140]:
from langchain.schema.agent import AgentFinish

In [157]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            output_text = result.return_values['output']
            return output_text
        tool = {
            "get_book_detail": get_book_detail, 
            "get_overall_temp": get_overall_temp,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [160]:
from IPython.display import display, Markdown

In [161]:
display(Markdown(run_agent("what is the weather is sf?")))

The weather in San Francisco is a sizzling 32°C, and it's sunny and windy. Don't forget your sunglasses and maybe a light jacket for that breeze! 🌞💨

In [ ]:
display(Markdown(run_agent("I want more information of book name Harry ")))

Books found:
ID: 1, Title: Harry potter, Price: 999.0


The book you're looking for is "Harry Potter." Here are the details:

- **Price:** 999.0 Baht
- **In Stock:** 47 copies available

So, if you're a wizard or just a fan of magic, you might want to grab one before they disappear! 🧙‍♂️✨